# Trip-Level Feature Engineering (GeoLife Project)

This notebook converts **GPS point-level data** into **trip-level analytical features**.

**Input:** Preprocessed GPS points (`gps_enriched.csv`)  
**Output:** Clean trip-level dataset suitable for analytics, dashboards, and ML

Each row in the final dataset represents **one complete trip**.

## Why Trip-Level Features?

Raw GPS data is too granular for most analytics tasks.
Decisions are made at the **trip level**, not the point level.

This step:
- Aggregates GPS points into trips
- Creates behaviorally meaningful features
- Applies strict data quality filters

Downstream dashboards and ML models depend entirely on the correctness of this step.


In [1]:
import pandas as pd

## Initial Sanity Checks

Before aggregation, we verify:
- Column names and data types
- Missing values
- Whether timestamps are correctly parsed

These checks prevent silent downstream errors.


In [2]:
df = pd.read_csv(
    "D:\projects\gps_project\Data\staging\gps_enriched.csv",
    parse_dates=["timestamp"]
)

df.head()

,lat,lon,altitude,timestamp,source_file,user_id,delta_time_s,large_gap,delta_dist_m,speed_mps,is_moving,trip_break,trip_id
0,39.984702,116.318417,492.0,2008-10-23 02:53:04,20081023025304.plt,0,0.0,False,0.000000,0.000000,False,False,0
1,39.984683,116.318450,492.0,2008-10-23 02:53:10,20081023025304.plt,0,6.0,False,3.516886,0.586148,True,False,0
2,39.984686,116.318417,492.0,2008-10-23 02:53:15,20081023025304.plt,0,5.0,False,2.831299,0.566260,True,False,0
3,39.984688,116.318385,492.0,2008-10-23 02:53:20,20081023025304.plt,0,5.0,False,2.735434,0.547087,True,False,0
4,39.984655,116.318263,492.0,2008-10-23 02:53:25,20081023025304.plt,0,5.0,False,11.023008,2.204602,True,False,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10776492 entries, 0 to 10776491
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   lat           float64       
 1   lon           float64       
 2   altitude      float64       
 3   timestamp     datetime64[ns]
 4   source_file   object        
 5   user_id       int64         
 6   delta_time_s  float64       
 7   large_gap     bool          
 8   delta_dist_m  float64       
 9   speed_mps     float64       
 10  is_moving     bool          
 11  trip_break    bool          
 12  trip_id       int64         
dtypes: bool(3), datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 853.0+ MB


In [4]:
df.isna().sum()

lat                 0
lon                 0
altitude            0
timestamp           0
source_file         0
user_id             0
delta_time_s        0
large_gap           0
delta_dist_m        0
speed_mps       15477
is_moving           0
trip_break          0
trip_id             0
dtype: int64

## Sort the Data

Trip aggregation assumes that GPS points are **temporally ordered**.
Even if earlier steps sorted the data, we do it again defensively.

This is a critical but often ignored step.


In [5]:
df = df.sort_values(["user_id", "trip_id", "timestamp"])


## Trip-Level Aggregation Logic

A **trip** is uniquely identified by:
- `user_id`
- `trip_id`

For each trip, we compute:
- Start and end time
- Duration
- Distance traveled
- Speed statistics
- Number of GPS points (data reliability indicator)


In [6]:
df.columns
df["distance_km"] = df["delta_dist_m"] / 1000
df["speed_kmph"] = df["speed_mps"] * 3.6


In [7]:
trip_df = (
    df
    .groupby(["user_id", "trip_id"])
    .agg(
        trip_start_time=("timestamp", "min"),
        trip_end_time=("timestamp", "max"),
        trip_duration_min=("timestamp", lambda x: (x.max() - x.min()).total_seconds() / 60),
        total_distance_km=("distance_km", "sum"),
        avg_speed_kmph=("speed_kmph", "mean"),
        max_speed_kmph=("speed_kmph", "max"),
        num_points=("timestamp", "count")
    )
    .reset_index()
)

trip_df.head()


,user_id,trip_id,trip_start_time,trip_end_time,trip_duration_min,total_distance_km,avg_speed_kmph,max_speed_kmph,num_points
0,0,0,2008-10-23 02:53:04,2008-10-23 03:05:15,12.183333,2.068493,10.065517,41.795366,148
1,0,1,2008-10-23 04:08:07,2008-10-23 04:34:52,26.750000,7.904753,9.963295,118.364809,322
2,0,2,2008-10-23 09:42:25,2008-10-23 10:33:20,50.916667,4.387869,5.389305,61.310087,403
3,0,3,2008-10-23 10:44:31,2008-10-23 10:46:11,1.666667,0.348248,7.450928,36.620371,22
4,0,4,2008-10-23 11:08:22,2008-10-23 11:11:12,2.833333,0.229258,6.783622,24.124796,13


In [8]:
df = df.drop(columns=["delta_dist_m", "speed_mps"])

## Interpreting Core Trip Features

- **trip_duration_min**: Temporal length of the trip  
- **total_distance_km**: Overall mobility intensity  
- **avg_speed_kmph**: Average movement speed  
- **num_points**: Proxy for data quality (low values are unreliable)

These form the backbone of all further analysis.


## Idle Time Ratio

Idle time ratio captures how much of the trip was spent nearly stationary.

Definition:
> Proportion of GPS points where speed < 1 km/h

This often captures congestion better than average speed.


In [9]:
idle_ratio = (
    df
    .assign(idle=lambda x: x["speed_kmph"] < 1)
    .groupby(["user_id", "trip_id"])["idle"]
    .mean()
    .reset_index(name="idle_time_ratio")
)

trip_df = trip_df.merge(
    idle_ratio,
    on=["user_id", "trip_id"],
    how="left"
)


## Spatial Spread (Bounding Box Area)

Bounding box area approximates how widely a trip is spatially distributed.

Interpretation:
- Small area + long duration → congestion or looping
- Large area + short duration → highway or fast transit


In [10]:
bbox = (
    df
    .groupby(["user_id", "trip_id"])
    .agg(
        min_lat=("lat", "min"),
        max_lat=("lat", "max"),
        min_lon=("lon", "min"),
        max_lon=("lon", "max")
    )
    .reset_index()
)

bbox["bounding_box_area"] = (
    (bbox["max_lat"] - bbox["min_lat"]) *
    (bbox["max_lon"] - bbox["min_lon"])
)

trip_df = trip_df.merge(
    bbox[["user_id", "trip_id", "bounding_box_area"]],
    on=["user_id", "trip_id"],
    how="left"
)


## Temporal Context Features

Simple binary flags provide strong contextual signals:
- Night trips may imply risk or logistics
- Weekday trips often indicate commuting behavior


In [11]:
trip_df["night_trip_flag"] = (
    trip_df["trip_start_time"].dt.hour
    .between(22, 5)
    .astype(int)
)

trip_df["weekday_flag"] = (
    trip_df["trip_start_time"].dt.weekday
    .between(0, 4)
    .astype(int)
)


## Trip Efficiency

Trip efficiency normalizes distance by time.

It allows comparison across trips of different lengths.


In [12]:
trip_df["trip_efficiency"] = (
    trip_df["total_distance_km"] / trip_df["trip_duration_min"]
)


## Data Quality Filters

Not all trips are analytically useful.

We remove:
- Extremely short trips
- Near-zero distance trips
- Unrealistic speeds
- Trips with insufficient GPS points


In [13]:
trip_df_clean = trip_df[
    (trip_df["trip_duration_min"].between(2, 300)) &
    (trip_df["total_distance_km"] > 0.1) &
    (trip_df["avg_speed_kmph"].between(1, 130)) &
    (trip_df["num_points"] >= 10)
].copy()


## Save Trip-Level Dataset

This dataset will be used for:
- SQL analytics (PostgreSql)
- Power BI dashboards
- Anomaly detection


In [18]:
trip_df_clean.to_csv(
    "D:\\projects\\gps_project\\Data\\trip_feature_clean.csv",
    index=False
)


In [20]:
trip_df_clean

,user_id,trip_id,trip_start_time,trip_end_time,trip_duration_min,total_distance_km,avg_speed_kmph,max_speed_kmph,num_points,idle_time_ratio,bounding_box_area,night_trip_flag,weekday_flag,trip_efficiency
0,0,0,2008-10-23 02:53:04,2008-10-23 03:05:15,12.183333,2.068493,10.065517,41.795366,148,0.013514,2.833498e-05,0,1,0.169781
1,0,1,2008-10-23 04:08:07,2008-10-23 04:34:52,26.750000,7.904753,9.963295,118.364809,322,0.074534,6.282003e-04,0,1,0.295505
2,0,2,2008-10-23 09:42:25,2008-10-23 10:33:20,50.916667,4.387869,5.389305,61.310087,403,0.146402,1.950538e-05,0,1,0.086177
4,0,4,2008-10-23 11:08:22,2008-10-23 11:11:12,2.833333,0.229258,6.783622,24.124796,13,0.076923,9.834440e-07,0,1,0.080914
5,0,5,2008-10-24 02:09:59,2008-10-24 02:29:26,19.450000,0.968690,3.323335,104.127482,176,0.448864,6.516863e-06,0,1,0.049804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17218,48,7,2009-09-03 22:35:55,2009-09-03 22:55:10,19.250000,4.533936,13.317565,73.696155,241,0.294606,3.939733e-04,0,1,0.235529
17219,48,8,2009-09-04 09:23:33,2009-09-04 12:24:09,180.600000,21.752508,8.033244,162.785709,1758,0.279295,1.120751e-03,0,1,0.120446
17220,48,9,2009-09-04 13:14:59,2009-09-04 17:21:38,246.650000,234.211703,58.264502,121.330934,2836,0.037377,1.634304e+00,0,1,0.949571
17222,49,1,2009-09-02 13:16:05,2009-09-02 15:15:05,119.000000,130.275915,65.462868,107.803633,1432,0.023743,6.226696e-01,0,1,1.094756


In [19]:
import pandas as pd

df = pd.read_csv(
    r"D:\projects\gps_project\Data\trip_feature_clean.csv"
)

len(df)

15240